# Data Cleaning in Natural Launguage Processing with Python


This notebook revists Pyohio2018 Natural Language Processing in Python
<br> Credit goes to https://github.com/adashofdata/nlp-in-python-tutorial

## Introduction

This notebook goes through a necessary step of any data science project - data cleaning. 

Data cleaning is a time consuming task, yet it's a very important one.
Without appropriately organised data, we will not be able to find right insight and answer the question.
If you are making a predictive model, feeding dirty data into a model will give us results that are meaningless.


Specifically, we'll be walking through:

1. **Getting the data - **this time, we'll be scraping data from a website
2. **Cleaning the data - **walk through popular text pre-processing techniques
3. **Organizing the data - **organize the cleaned data into a way that will be easy to input into other algorithms

The output of this notebook will be clean, organized data in two standard text formats:

1. **Corpus** - a collection of text
2. **Document-Term Matrix** - word counts in matrix format

## Objective

Our goal, this time, is to look at transcripts of various comedians and note their similarities and differences.
Specifically, we'd like to know if Ali Wong's comedy style is different than other comedians: as an expamle https://www.youtube.com/watch?v=u4lG9jWn9pU

## Data Collection

Before we clean data, we need to get data.
<br>Luckily, there are wonderful people online that keep track of stand up routine transcripts. [Scraps From The Loft](http://scrapsfromtheloft.com) makes them available for non-profit and educational purposes.

To decide which comedians to look into (as comparisions of Ali Wong), let's check IMDB and see specifically at comedy specials that were released in the past 5 years. To narrow it down further, let's only look at those with greater than a 7.5/10 rating and more than 2000 votes. If a comedian had multiple specials that fit those requirements, pick the most highly rated one.

In [ ]:
# Web scraping, pickle imports
import requests                #Make HTTP requests and get information from a website
from bs4 import BeautifulSoup  #Parse HTML documents and extract parts of a website
import pickle                  #Serialise Python objects and save data for later

# Scrapes transcript data from scrapsfromtheloft.com
def url_to_transcript(url):
    '''Returns transcript data from scrapsfromtheloft.com.'''
    page = requests.get(url).text      
    #use 'request' to get all data from url, and Fetch the returned response body in text format
    
    soup = BeautifulSoup(page, "lxml") 
    #tell python that the text fetched is HTML format 

    text = [p.text for p in soup.find(class_="post-content").find_all('p')] 
    #extract the texts in <div class="post-content"> of the website, and find all the paragraphs in there.
    
    print(url)
    return text

# List URLs of transcripts in scope
urls = ['http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/',
        'http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/',
        'http://scrapsfromtheloft.com/2018/03/15/ricky-gervais-humanity-transcript/',
        'http://scrapsfromtheloft.com/2017/08/07/bo-burnham-2013-full-transcript/',
        'http://scrapsfromtheloft.com/2017/05/24/bill-burr-im-sorry-feel-way-2014-full-transcript/',
        'http://scrapsfromtheloft.com/2017/04/21/jim-jefferies-bare-2014-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/02/john-mulaney-comeback-kid-2015-full-transcript/',
        'http://scrapsfromtheloft.com/2017/10/21/hasan-minhaj-homecoming-king-2017-full-transcript/',
        'http://scrapsfromtheloft.com/2017/09/19/ali-wong-baby-cobra-2016-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/03/anthony-jeselnik-thoughts-prayers-2015-full-transcript/',
        'http://scrapsfromtheloft.com/2018/03/03/mike-birbiglia-my-girlfriends-boyfriend-2013-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/19/joe-rogan-triggered-2016-full-transcript/']

# Comedian names
comedians = ['louis', 'dave', 'ricky', 'bo', 'bill', 'jim', 'john', 'hasan', 'ali', 'anthony', 'mike', 'joe']

In [ ]:
# Actually request transcripts (takes a few minutes to run)
# Returns transcript data specifically from scrapsfromtheloft.com.
transcripts = [url_to_transcript(u) for u in urls]

In [ ]:
# Pickle files for later use

# Make a new directory to hold the text files
# !mkdir transcripts

for i, c in enumerate(comedians):
     with open("transcripts/" + c + ".txt", "wb") as file: # open(file name[, access mode][, buffersize])
         pickle.dump(transcripts[i], file) # Python's pickle module is to serialize objects so they can be saved to a file, and loaded in a program again later on.

In [ ]:
# Load pickled files
data = {}
for i, c in enumerate(comedians):
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [ ]:
# Double check to make sure the data has been loaded properly
data.keys()

In [ ]:
# More checks
data['louis'][:3]

## Text Data Cleaning 

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. 

With text data, there are some common data cleaning techniques.

This cleaning process can go on forever. There's always an exception to every cleaning step and approach. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Below are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common text data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text (dividing big text data into smaller parts called tokens)
* Remove stop words (stop word is a commonly used word (such as “the”, “a”, “an”, “in”). Search engine has been programmed to ignore those.)

**More text data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

Let's take a look at our data again.
<br>The iter() function creates an object which can be iterated one element at a time.
<br>The object is useful when coupled with loops like for loop, while loop.
<br>The syntax of the iter() function is: iter(object, sentinel(as (optional)) )
<br>Each time you call iter() on the data loader, a new iterator is generated. 

In [ ]:
next(iter(data.keys()))

Notice that our dictionary is currently in key: comedian, value: list of text format

In [ ]:
next(iter(data.values()))

In [ ]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [ ]:
# Combine it!
# items() method is used to return the list with all dictionary keys with values.
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [ ]:
data_combined

In [ ]:
# We put the dictionary into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',200)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['Transcript'] #set column labels of the DataFrame
data_df = data_df.sort_index() #sort
data_df

In [ ]:
# Let's take a look at the transcript for Ali Wong
'''DataFrame.loc = Access a group of rows and columns by label(s) or a boolean array.'''
data_df.Transcript.loc['ali']

## Apply a first round of text cleaning techniques

In [ ]:
import re # RegEx can be used to check if a string contains the specified search pattern.
import string

def clean_text_round1(text):
    '''Make text lowercase, 
       remove text in square brackets, 
       remove punctuation and 
       remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    '''/ = delimiter
       .* = zero or more of anything but newline
       \S = anything except a whitespace (newline, tab, space)'''
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation from a string
    text = re.sub('\w*\d\w*', '', text) # \d matches [0-9] and other digit characters, for example Eastern Arabic numerals ٠١٢٣٤٥٦٧٨٩.
    return text

round1 = lambda x: clean_text_round1(x)

The % Operator: https://stackabuse.com/python-string-interpolation-with-the-percent-operator/

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.Transcript.apply(round1))
data_clean

## Apply a second round of cleaning

In [ ]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.Transcript.apply(round2))
data_clean

**NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:
* Mark 'cheering' and 'cheer' as the same word (stemming / lemmatization)
* Combine 'thank you' into one term (bi-grams)
* And a lot more...

## Organizing The Data

I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:
1. **Corpus - **a collection of text
2. **Document-Term Matrix - **word counts in matrix format

### Corpus

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [ ]:
# Let's take a look at our dataframe
data_df

In [ ]:
# Let's add the comedians' full names as well
full_names = ['Ali Wong', 'Anthony Jeselnik', 'Bill Burr', 'Bo Burnham', 'Dave Chappelle', 'Hasan Minhaj',
              'Jim Jefferies', 'Joe Rogan', 'John Mulaney', 'Louis C.K.', 'Mike Birbiglia', 'Ricky Gervais']

# Using 'Full_name' as the column name and equating it to the list 
data_df['Full_name'] = full_names
data_df

In [ ]:
# Let's pickle it for later use
data_df.to_pickle("corpus.pkl")

## Document-Term Matrix
https://en.wikipedia.org/wiki/Document-term_matrix

After we cleaned our data, we will now transform our corpus to a numerical format to make it understandable by the machine.
<br>For many of the techniques we'll be using, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words (splitting sentences and words from the body of the text).
<br>We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [ ]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.Transcript)
'''By the fit method, when applied to the training dataset, learns the model parameters (for example, mean and standard deviation). 
   We then need to apply the transform method on the training dataset to get the transformed (scaled) training dataset.
   If both these data are the same (i.e. the data for calculating the means and the data that means are applied to) 
   you can use fit_transform which is basically a fit followed by a transform.
   further: https://datascience.stackexchange.com/questions/12321/difference-between-fit-and-fit-transform-in-scikit-learn-models'''
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names()) # Convert to Panda DataFrame for human readability
data_dtm.index = data_clean.index
data_dtm

In [ ]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [ ]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))